# Contents
Supporting information for Supplementary Table 3 in "Life cycle comparison of multi-step lithium-ion battery recycling processes to conventional mining refinement", by Michael L. Machala, Xi Chen, Samantha P. Bunke, Gregory Forbes, Akarys Yegizbay, Jacques A. de Chalendar, Inês L. Azevedo, Sally M. Benson, and William A. Tarpeh.
1. Generate 2019 grid mix in CISO, NEVP, WACM, BPAT, US
2. Compute consumption-based environmental impact factors in 2019 for the same balancing areas. Technology specific environmental impact factors are sourced from `technology EFs.csv` 

To reproduce these calculations,
* The [`gridemissions` package](https://github.com/jdechalendar/gridemissions) needs to be available to your Python installation,
* `"EBA_elec.csv"` needs to be downloaded from [energy.stanford.edu/gridemissions](energy.stanford.edu/gridemissions) and stored at `gridemissions.config["DATA_PATH"]`.
See the README for the `gridemissions` repository for how to configure your `DATA_PATH`.

In [ ]:
import sys

print(sys.executable)

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd

import gridemissions
from gridemissions.load import BaData
from gridemissions.viz.reports import cleaning_plot
from gridemissions import eia_api
from gridemissions.emissions import BaDataEmissionsCalc

In [ ]:
gridemissions.config["DATA_PATH"]

# 1. 2019 grid mix in CISO, NEVP, WACM, BPAT, US

In [ ]:
file_name_elec = gridemissions.config["DATA_PATH"] / "EBA_elec.csv"

elec = BaData(fileNm=file_name_elec)

start = pd.to_datetime("20190101T00Z")
end = pd.to_datetime("20200101T00Z")
bas = ["CISO", "NEVP", "WACM", "BPAT"]


def get_grid_mix(ba):
    NG_cols = sum((elec.get_cols(r=ba, field=f"SRC_{src}") for src in eia_api.SRC), [])
    NG_cols = [c for c in NG_cols if c in elec.df.columns]
    grid_mix_2019 = elec.df.loc[start:end, NG_cols].sum()
    grid_mix_2019.index = grid_mix_2019.index.map(lambda x: x.split(".")[3])
    # grid_mix_2019 = grid_mix_2019 / grid_mix_2019.sum()
    return grid_mix_2019


grid_mixes_all = pd.DataFrame({ba: get_grid_mix(ba) for ba in elec.regions}).fillna(0.0)
grid_mix_us = grid_mixes_all.sum(axis=1)
grid_mix_us /= grid_mix_us.sum()

grid_mixes = grid_mixes_all[bas].copy()
grid_mixes /= grid_mixes.sum()
grid_mixes.loc[:, "US"] = grid_mix_us
grid_mixes.loc[
    ["WAT", "NUC", "SUN", "NG", "WND", "COL", "OIL", "BIO", "GEO", "OTH"]
].to_csv("2019_grid_mixes.csv")

# 2. Consumption-based environmental impact factors in 2019

In [ ]:
file_name_elec = gridemissions.config["DATA_PATH"] / "EBA_elec.csv"
file_name_co2 = gridemissions.config["DATA_PATH"] / "EBA_co2.csv"

elec = BaData(fileNm=file_name_elec)
co2 = BaData(fileNm=file_name_co2, variable="CO2")

In [ ]:
start = pd.to_datetime("20190101T00Z")
end = pd.to_datetime("20200101T00Z")

# Hack: Remove CFE from dataset before this date
CFE_cutoff_date = pd.to_datetime("20210128T20Z")
non_CFE_cols = [c for c in elec.df.columns if not (("CFE-" in c) or ("-CFE" in c))]

# Remove all columns with NaNs?
cols = [
    c
    for c in non_CFE_cols
    if c not in ["EBA.GRID-ALL.NG.OTH.H", "EBA.IPCO-ALL.NG.COL.H"]
]

if (start < CFE_cutoff_date) and (end < CFE_cutoff_date):
    mini_elec = BaData(df=elec.df.loc[start:end, cols])
elif (start < CFE_cutoff_date) and (end > CFE_cutoff_date):
    raise ValueError("Not supported")
else:
    mini_elec = BaData(df=elec.df.loc[start:end])

In [ ]:
EFs = pd.read_csv("technology EFs.csv", index_col=0).fillna(0.0)
EFs.loc["UNK"] = EFs.loc["OTH"]

In [ ]:
def get_consumption_efs(poll):
    co2_calc = BaDataEmissionsCalc(mini_elec, poll=poll, EF=EFs[poll])
    co2_calc.process()

    poll_data = co2_calc.poll_data
    polli_data = pd.DataFrame(
        {
            ba: (
                poll_data.df.loc[
                    start:end, poll_data.get_cols(ba, field="D")
                ].values.flatten()
                / elec.df.loc[start:end, elec.get_cols(ba, field="D")].values.flatten()
            )
            for ba in poll_data.regions
        },
        index=poll_data.df.loc[start:end].index,
    )

    return polli_data[bas].mean()


pd.DataFrame({poll: get_consumption_efs(poll) for poll in EFs.columns}).to_csv(
    "2019 consumption EFs.csv"
)